### Kernel regression and splines
https://www.ics.uci.edu/~welling/classnotes/papers_class/Kernel-Ridge.pdf. The choice of kernel implicitly selects the basis functions selected for kernel expansion. With a polynomial basis, we can fit Lagrange polynomials.

In [ ]:
x = np.random.randn(2)
y = np.random.randn(1, 2) + 4.
def phi(x):
    x = x[None, :]
    return np.concatenate((np.ones_like(x), x, x**2, x**3, x**4, x**5, x**6, x**7, x**8), axis=0)

w, f = utils.signal.basis_function_regression(x, y, phi, lambd=0.)

x_eval = np.linspace(x.min(), x.max(), 100)
ph = phi(x_eval)[None, ...] # o, d, n
f = (w[..., None]*ph).sum(1) # o, n

plt.plot(x_eval, f[0, :])
plt.scatter(x, y[0, :])
plt.show()

In [ ]:
x = torch.linspace(0, 6, 7)
y = x.sin()
xs = torch.linspace(0, 6, 101)

spline = utils.signal.spline_interpolate(3)

ys = spline.interp(x, y, xs)
Ys = spline.integ(x, y, xs)

plt.scatter(x, y, label='Samples', color='purple')
plt.plot(xs, ys, label='Interpolated curve')
plt.plot(xs, xs.sin(), '--', label='True Curve')
plt.plot(xs, Ys, label='Spline Integral')
plt.plot(xs, 1-xs.cos(), '--', label='True Integral')
plt.legend()
plt.show()

In [ ]:
linn = utils.signal.linear_interpolate(x.numpy(), y.numpy(), xs.numpy())
plt.plot(xs.numpy(), linn)
plt.scatter(x.numpy(), y.numpy())

### Product model

In [2]:
# Synthetic data
sample_bin, track_samples, x_t, y_t, s_t, hd_t, theta_t, dir_t, \
    syn_t_spike, spike_samples, neurons, \
    left_x, right_x, bottom_y, top_y = pickle.load(open('./data/gauss_spat_IPP.p', 'rb'))

fac = 1.

max_speed = s_t.max()/fac
wrap_theta_t = utils.numpy.WrapPi(theta_t, True)

right_x = right_x/fac
left_x = left_x/fac
top_y = top_y/fac
bottom_y = bottom_y/fac

arena_width = right_x - left_x
arena_height = top_y - bottom_y
behav_data = [x_t/fac, y_t/fac]


bin_size = 1
tbin, resamples, rc_t, rbehav_t = utils.neural.BinTrain(bin_size, sample_bin, syn_t_spike, 
                                                 track_samples, behaviour_data=behav_data, binned=False)


In [16]:
def GP_(neurons):
    """
    GP with variable regressors model fit and nonconvexity.
    
    Here active dimensions are handled by the GP kernels, None entries indicate non-active dimensions.
    """
    num_induc = 16
    l = 100.*np.ones(neurons)
    v = np.ones(neurons)

    ind_list = [np.linspace(left_x, right_x, num_induc), \
                (bottom_y + arena_height*np.random.rand(num_induc))]

    kt = [('variance', v), (None,), ('RBF', 'euclid', np.array([l]))]
    inducing_points = np.array(ind_list).T[None, ...].repeat(neurons, axis=0)
    gp_rate = mdl.nonparametrics.Gaussian_process(
        len(ind_list), neurons, kt, inducing_points=inducing_points, jitter=1e-5, 
        inv_link='exp', mean=np.zeros((neurons)), learn_mean=True
    )
    
    kt = [('variance', v), ('RBF', 'euclid', np.array([l])), (None,)]
    inducing_points = np.array(ind_list).T[None, ...].repeat(neurons, axis=0)
    gp_rate_ = mdl.nonparametrics.Gaussian_process(
        len(ind_list), neurons, kt, inducing_points=inducing_points, jitter=1e-5, 
        inv_link='exp', mean=np.zeros((neurons)), learn_mean=True
    )
    
    rate_model = mdl.parametrics.product_model(2, [gp_rate, gp_rate_], inv_link='relu')
    return rate_model



def GP(neurons):
    """
    GP with variable regressors model fit and nonconvexity.
    
    Here active dimensions are handled by the GP kernels, None entries indicate non-active dimensions.
    """
    num_induc = 16
    l = 100.*np.ones(neurons)
    v = np.ones(neurons)

    ind_list = [np.linspace(left_x, right_x, num_induc)]
    kt = [('variance', v), ('RBF', 'euclid', np.array([l]))]
    inducing_points = np.array(ind_list).T[None, ...].repeat(neurons, axis=0)
    gp_rate = mdl.nonparametrics.Gaussian_process(
        len(ind_list), neurons, kt, inducing_points=inducing_points, jitter=1e-5, 
        inv_link='exp', mean=np.zeros((neurons)), learn_mean=True, active_dims=[0]
    )
    
    ind_list = [(bottom_y + arena_height*np.random.rand(num_induc))]
    kt = [('variance', v), ('RBF', 'euclid', np.array([l]))]
    inducing_points = np.array(ind_list).T[None, ...].repeat(neurons, axis=0)
    gp_rate_ = mdl.nonparametrics.Gaussian_process(
        len(ind_list), neurons, kt, inducing_points=inducing_points, jitter=1e-5, 
        inv_link='exp', mean=np.zeros((neurons)), learn_mean=True, active_dims=[1]
    )
    
    rate_model = mdl.parametrics.product_model(2, [gp_rate, gp_rate_], inv_link='relu')
    return rate_model

In [19]:
inputs = mdl.inference.input_group(2, [(None, None, None, 1)]*2)

gp_1 = GP_(neurons)

# Poisson process output
likelihood = mdl.likelihoods.Poisson(tbin, neurons, 'relu')

# NLL model
hist_len = 1
samples = resamples
glm = mdl.inference.VI_optimized(inputs, gp_1, likelihood)
glm.set_data([rbehav_t[0][:samples], rbehav_t[1][:samples]], samples, rc_t[:, :samples], batch_size=100000)
glm.to(dev)

# fit
sch = lambda o: optim.lr_scheduler.MultiplicativeLR(o, lambda e: 0.9)
opt_tuple = (optim.Adam, 100, sch)
opt_lr_dict = {'default': 1e-4}
glm.set_optimizers(opt_tuple, opt_lr_dict)

annealing = lambda x: 1.0#min(1.0, 0.002*x)
losses = glm.fit(1000, loss_margin=1e0, stop_iters=100, anneal_func=annealing, cov_samples=1, ll_samples=10)


plt.figure()
plt.plot(losses)
plt.xlabel('epoch')
plt.ylabel('NLL')
plt.show()

KeyboardInterrupt: 

In [ ]:
# show fields
show_neurons = [4]
fig, axes = plt.subplots(1, 1, figsize=(5,4))

grid_size = [50, 40]
grid_shape = [[left_x, right_x], [bottom_y, top_y]]

ticktitle='firing rate (Hz)'

def func(pos):
    prevshape = pos.shape[1:]
    x = pos[0].flatten()
    y = pos[1].flatten()
    #theta = np.zeros(len(x))
    covariates = np.array([x, y])#, theta])
    return gp_1.eval_rate(covariates, 0).reshape(*prevshape)

_, field = tools.compute_mesh(grid_size, grid_shape, func)

_, ax = tools.visualize_field(field, grid_shape, ticktitle=ticktitle, spike_pos=None, figax=(fig, axes))
x = np.linspace(left_x, right_x, 10)
#tools.decorate_ax(ax, xlim=[left_x, right_x], ylim=[bottom_y, top_y])}

plt.show()

In [ ]:
left_x = 0.0
right_x = 538.4177999999999
bottom_y = 0.0
top_y = 432.3225
arena_width = right_x - left_x
arena_height = top_y - bottom_y



def generate_data(track_samples=3000000):
    neurons = 1
    sample_bin = 1./1250
    
    # Mixture of Gaussians
    t_p = np.array([[0.0, 0.0]])
    hd_p = np.array([[0.0, 0.0]])
    dir_p = np.array([[0.0, 0.0]])
    sp = np.array([0.0])
        
    mu = np.array([[100., 100.]])
    prec = np.array([[0.0002, 0.0002, 0.0]])
    rate_0 = np.array([10.0]) # Hz
    
    gauss_1 = mdl.parametrics.PTP_field(neurons, 'exp')
    gauss_1.set_params(sample_bin, mu, prec, rate_0, t_p, hd_p, dir_p, sp)

    mu = np.array([[170., 350.]])
    prec = np.array([[0.0002, 0.0002, 0.0]])
    rate_0 = np.array([10.0]) # Hz

    gauss_2 = mdl.parametrics.PTP_field(neurons, 'exp')
    gauss_2.set_params(sample_bin, mu, prec, rate_0, t_p, hd_p, dir_p, sp)

    mu = np.array([[400., 225.]])
    prec = np.array([[0.0003, 0.0001, 0.0]])
    rate_0 = np.array([10.0]) # Hz
    
    gauss_3 = mdl.parametrics.PTP_field(neurons, 'exp')
    gauss_3.set_params(sample_bin, mu, prec, rate_0, t_p, hd_p, dir_p, sp)

    gauss_rate = mdl.parametrics.mixture_model([gauss_1, gauss_2, gauss_3])
    gauss_rate.set_params(sample_bin)
    
    # sample behaviour
    print('Generating animal behaviour...')
    arena = ['box', (left_x, right_x, bottom_y, top_y)]
    an = mdl.animal.animal_SL(sample_bin, track_samples, arena)

    sample_bin, sim_samples, x_t, y_t, \
        s_t, dir_t, hd_t, theta_t = \
        an.sample(0.01, [200.0, 200.0], 200, 0.14, 0.0)

    wrap_theta_t = tools.WrapPi(theta_t, True)
    maxspeed = s_t.max()
    
    behav_data = [x_t, y_t, s_t, wrap_theta_t, hd_t, dir_t]
    
    # add true field
    unit = 0
    grid_size = [int(arena_width/10), int(arena_height/10)]
    grid_shape = [[left_x, right_x], [bottom_y, top_y]]

    def func(pos):
        prevshape = pos.shape[1:]
        x = pos[0].flatten()
        y = pos[1].flatten()
        s = np.zeros_like(x)
        th = np.zeros_like(x)
        hd = np.zeros_like(x)
        dr = np.zeros_like(x)
        covariates = [x, y, s, th, hd, dr]
        return gauss_rate.eval_rate(covariates, unit).reshape(*prevshape)

    print('Computing true field...')
    true_field = tools.compute_mesh(grid_size, grid_shape, func)[1]
    #_, ax = tools.visualize_field(true_field, grid_shape, ticktitle='')
    #plt.show()
    
    # sample spikes
    #shape = np.ones(neurons)
    #renewal_dist = mdl.likelihoods.Gamma(neurons, 'exp', shape)
    renewal_dist = mdl.likelihoods.Poisson(neurons, 'exp')
    renewal_dist.set_params(sample_bin)
    
    glm = mdl.inference.nll_optimized([gauss_rate], renewal_dist)
    glm.to(dev)
    
    print('Sampling spike trains...')
    ini_train = np.zeros((1, 1, 1)) # for trials
    rc_t, _ = glm.sample(behav_data, ini_train)
    
    return true_field, behav_data, rc_t[0], sample_bin, maxspeed


In [ ]:
# setup GP parameters
def GP_params(behav_tuple, num_induc, maxspeed):
    units_ = 1
    l = np.ones(units_)
    v = np.ones(units_)

    ind_list = [np.linspace(left_x, right_x, num_induc), \
                bottom_y + arena_height*np.random.rand(num_induc)]

    kt = [('RBF', 'euclid', np.array([l, l]), v)]
    covariates = (behav_tuple[0], behav_tuple[1])
        
    return covariates, kt, ind_list, units_

In [ ]:
# Synthetic multimodal data
true_field, behav_data, rc_t, sample_bin, maxspeed = generate_data()
units = 1

In [ ]:
fac = 1000.
behav_list = [b / fac for b in behav_data[:2]]

t_spike = []
for binned in rc_t:
    t_spike.append(neural_utils.BinToTrain(binned))

In [ ]:
def get_field(glm_rate, grid_size, grid_shape, neuron):

    def func(pos):
        prevshape = pos.shape[1:]
        x = pos[0].flatten()
        y = pos[1].flatten()
        covariates = [x, y]
        return glm_rate.eval_rate(covariates, neuron).reshape(-1, *prevshape)

    _, field = tools.compute_mesh(grid_size, grid_shape, func)
    return field


def compute_rate(glm_rate, neuron):
    grid_size = [int(arena_width/10), int(arena_height/10)]
    grid_shape = [[left_x/fac, right_x/fac], [bottom_y/fac, top_y/fac]]
    field = get_field(glm_rate, grid_size, grid_shape, neuron)
    return field


def compute_RMS(true_w, model_w): # rate error, with shape (neuron,)
    return np.sqrt(((model_w-true_w)**2).mean())
    
    
def compute_loss(model, w_1, w_2, range_w_1, range_w_2):
    r"""
    :param tuple w_i: tuple containing (parameter, index)
    """
    lw_1 = w_1[0].data[w_1[1]].item()
    lw_2 = w_2[0].data[w_2[1]].item()

    def func(pos):
        prevshape = pos[0, ...].shape
        p_1 = pos[0, ...].flatten()
        p_2 = pos[1, ...].flatten()
        NLL = np.empty(p_1.shape)
        with torch.no_grad():
            for k in range(len(p_1)): 
                w_1[0].data[w_1[1]] = torch.tensor(p_1[k], device=w_1[0].device)
                w_2[0].data[w_2[1]] = torch.tensor(p_2[k], device=w_1[0].device)
                NLL[k] = model.nll(0, ll_samples=10).item()
        return NLL.reshape(*prevshape)
    
    grid_size = [len(range_w_1), len(range_w_2)]
    grid_shape = [[range_w_1.min(), range_w_1.max()], [range_w_2.min(), range_w_2.max()]]
    _, landscape = tools.compute_mesh(grid_size, grid_shape, func)
    
    w_1[0].data[w_1[1]] = lw_1
    w_2[0].data[w_2[1]] = lw_2
    return landscape

In [ ]:
def mixture():# Mixture of Gaussians
    mu = np.array([[100., 100.]])/fac
    prec = np.array([[0.0002, 0.0002, 0.0]])*fac**2
    rate_0 = np.array([10.0]) # Hz
    t_p = np.array([[1.0, 0.2]]) # beta, phi_0 for theta modulation
    neurons = rate_0.shape[0]

    gauss_1 = mdl.parametrics.Gauss_GLM(neurons, 'exp')
    gauss_1.set_params(sample_bin, mdl.parametrics.gaussian_to_w(mu, prec, rate_0, t_p)[:, :6])

    mu = np.array([[170., 350.]])/fac
    prec = np.array([[0.0002, 0.0002, 0.0]])*fac**2
    rate_0 = np.array([10.0]) # Hz
    t_p = np.array([[1.0, 0.2]]) # beta, phi_0 for theta modulation
    neurons = rate_0.shape[0]

    gauss_2 = mdl.parametrics.Gauss_GLM(neurons, 'exp')
    gauss_2.set_params(sample_bin, mdl.parametrics.gaussian_to_w(mu, prec, rate_0, t_p)[:, :6])

    mu = np.array([[400., 225.]])/fac
    prec = np.array([[0.0003, 0.0001, 0.0]])*fac**2
    rate_0 = np.array([10.0]) # Hz
    t_p = np.array([[1.0, 0.2]]) # beta, phi_0 for theta modulation
    neurons = rate_0.shape[0]

    gauss_3 = mdl.parametrics.Gauss_GLM(neurons, 'exp')
    gauss_3.set_params(sample_bin, mdl.parametrics.gaussian_to_w(mu, prec, rate_0, t_p)[:, :6])

    glm_rate = mdl.parametrics.mixture_model([gauss_1, gauss_2, gauss_3])
    glm_rate.set_params(sample_bin)
    
    return glm_rate


def GP():# GP with variable regressors model fit and nonconvexity
    num_induc = 16
    l = 10.*np.ones(units)/fac
    v = np.ones(units)

    ind_list = [np.linspace(left_x, right_x, num_induc)/fac, \
                (bottom_y + arena_height*np.random.rand(num_induc))/fac]

    kt = [('RBF', 'euclid', np.array([l, l]), v)]

    inducing_points = np.array(ind_list).T[None, ...].repeat(1, axis=0)
    glm_rate = mdl.nonparametrics.GP_covariates(units, inducing_points, kt,
                                                ([None],)*2, ([None],)*2, 
                                                inv_link='exp', shared_kernel_params=True,
                                                full_cov_fit=False, 
                                                mean_ini=np.zeros((1, units, 1)), jitter=1e-5)
    glm_rate.set_params(sample_bin)
    return glm_rate


In [ ]:
# sequential data increase fitting
track_samples = rc_t.shape[1]
use_samples = [100000, 300000, 500000, 1000000, 1500000, 2000000, track_samples]

RMS = []
field_tuples = []

for model_type in ['mix', 'GP']:
    if model_type == 'mix':
        glm_rate = mixture()
        lr = 5*1e-3
    elif model_type == 'GP':
        glm_rate = GP()
        lr = 1e-3

    for samples in use_samples:
        rc_t_ = rc_t[:, :samples]
        #if model_type == 'GP' and samples < 1000000:
        #    lr = 1e-2
    
        # Poisson process output
        likelihood = mdl.likelihoods.Poisson(units, 'exp')
        likelihood.set_params(sample_bin)
    
        glm = mdl.inference.nll_optimized([glm_rate], likelihood)
        glm.preprocess([c[:samples] for c in behav_list], samples, rc_t_, batch_size=100000)
        glm.to(dev)

        # fit
        sch = lambda o: optim.lr_scheduler.MultiplicativeLR(o, lambda e: 0.9)
        opt_tuple = (optim.Adam, 100, sch)
        opt_lr_dict = {'default': lr}
        glm.set_optimizers(opt_tuple, opt_lr_dict)

        annealing = lambda x: 1.0#min(1.0, 0.002*x)
        losses = glm.fit(3000, margin=1e0, premature=100, anneal_func=annealing, cov_samples=1, ll_samples=10, pred_ll=False)

        plt.figure()
        plt.plot(losses)
        plt.xlabel('epoch')
        plt.ylabel('NLL')
        plt.show()
        
        field = compute_rate(glm_rate, [0])[0] # field computation
        # rate error
        RMS.append(compute_RMS(true_field, field))
        
        if model_type == 'GP':
            field_tuples.append(field)


In [ ]:
# loss landscape for GP
w_1 = (glm.rate_model[0].Xu, (0, 0, 0)) # u_x
w_2 = (glm.rate_model[0].Xu, (0, 0, 1)) # u_y
#w_1 = (glm.rate_model[0].models[0].w, (0, 1))
#w_2 = (glm.rate_model[0].models[0].w, (0, 2))

lw_1 = w_1[0].data[w_1[1]].item()
lw_2 = w_2[0].data[w_2[1]].item()
range_w_1 = np.linspace(lw_1-0.15, lw_1+0.02, 20)
range_w_2 = np.linspace(lw_2-0.01, lw_2+0.06, 20) 

resamples = rc_t.shape[1]
glm.preprocess(behav_list, resamples, rc_t, batch_size=resamples) # evaluate NLL for all data
landscape = compute_loss(glm, w_1, w_2, range_w_1, range_w_2)

lw_x = lw_1
lw_y = lw_2

In [ ]:
fig = plt.figure(figsize=(8,3))
fig.text(-0.03, 1.1, 'A', transform=fig.transFigure, size=15)
fig.text(-0.03, 0.53, 'B', transform=fig.transFigure, size=15)
fig.text(0.73, 0.51, 'C', transform=fig.transFigure, size=15)
runs = len(use_samples)


### fitting sequence
grid_size = [50, 40]
grid_shape = [[left_x, right_x], [bottom_y, top_y]]

# GT
widths = [1]
heights = [1]
nrows = 1
ncols = 1
spec = fig.add_gridspec(ncols=ncols, nrows=nrows, width_ratios=widths,
                         height_ratios=heights, left=0., right=.2, bottom=0.6, top=1.0)

ax = fig.add_subplot(spec[0, 0])
ax.set_title('{:.1f} Hz'.format(true_field.max()), fontsize=12)
_, ax = tools.visualize_field(true_field, grid_shape, figax=(fig, ax), ticks=[0, true_field.max()], 
                              ticklabels=['0', 'max'], ticktitle='firing rate')

ax.text(0.5, 1.3, 'true model', transform=ax.transAxes, fontsize=12, ha='center')
ax.set_title('{:.1f} Hz'.format(true_field.max()), fontsize=12)
ax.annotate('fits with increasing data', xy=(3.9, 1.30), xytext=(3.9, 1.40), xycoords='axes fraction', 
            fontsize=12, ha='center', va='bottom',
            arrowprops=dict(arrowstyle='-[, widthB=17.0, lengthB=0.2', lw=2.0))

# plot fields
widths = [1, 1, 1]
heights = [1]
nrows = 1
ncols = 3
spec = fig.add_gridspec(ncols=ncols, nrows=nrows, width_ratios=widths,
                         height_ratios=heights, wspace=0.25, left=0.3, right=1., bottom=0.6, top=1.0)

x_t = behav_data[0]
y_t = behav_data[1]

for k in range(3):
    samp = use_samples[k]
    t_spike_ = []
    for tt in t_spike:
        t_spike_.append(tt[tt < samp])
    x_s, y_s = neural_utils.CovariatesAtSpikes(t_spike_, (x_t, y_t))
    st = (x_s[0][::4], y_s[0][::4])

    ax = fig.add_subplot(spec[0, k])
    ax.set_title('{:.1f} Hz'.format(field_tuples[k].max()), fontsize=12)
    _, ax = tools.visualize_field(field_tuples[k], grid_shape, spike_pos=st, figax=(fig, ax), ticktitle='', cbar=False)



### RMS versus amount of data, loss
widths = [3, 1]
heights = [1]
nrows = 1
ncols = 2
spec = fig.add_gridspec(ncols=ncols, nrows=nrows, width_ratios=widths,
                         height_ratios=heights, left=0.05, right=1., bottom=0., top=.52)

ax = fig.add_subplot(spec[0, 0])

ax.plot(np.log10(use_samples), RMS[:runs], marker='o', label='parametric')
ax.plot(np.log10(use_samples), RMS[runs:], marker='o', label='GP')
ax.set_ylabel('RMS error (Hz)')
ax.set_xlabel('time steps in data')
ax.set_ylim(0.)
ax.set_xlim(0.)
ax.ticklabel_format(axis='y', style='sci', scilimits=(-4, 4))
ax.legend()

# loss landscape for GP
ax = fig.add_subplot(spec[0, 1])
grid_shape = [[range_w_1[0], range_w_1[-1]], [range_w_2[0], range_w_2[-1]]]

_, ax = tools.visualize_field(landscape, grid_shape, ticktitle='loss', figax=(fig, ax), ticks=[], 
                              vmin=None, cmap='plasma', cbar_format='%.1e', aspect='auto')

ax.scatter(lw_x, lw_y, marker='+', color='w', s=100)
#ax.set_xticks(list(range_w_1[::3]))
#ax.set_yticks(list(range_w_2[::3]))
ax.set_xlabel(r'$u_{1x}$')
ax.set_ylabel(r'$u_{1y}$')



plt.savefig('output/fit_sequence.svg')
plt.savefig('output/fit_sequence.pdf')
plt.show()